### Concept of the app
1. Get Articles (once, weekly, daily, ...)

    a. scraping & parsing
    
    b. API access

    c. save to DB!!!

2. Recognise keywords and main topic
    
    a. TF-IDF
    
    b. ???

3. Group with other of the same topic
    
    a. Main topic matching
    
    b. Hierarchy (decision tree?)

4. Find divergences

    a. diff on keyword set

    b. difference of opinion

    c. difference of data sources

    d. difference in 

5. Represent articles and divergences

In [ ]:
import os
import sys
import time
import lxml
import json
import shutil
import codecs
import requests
import psycopg2
import sqlalchemy
import urllib.parse

import numpy as np
import pandas as pd

from random import random
from datetime import datetime
from datetime import timezone
from selenium.webdriver import Chrome

In [ ]:
webdriver_path = './chromedriver_v84'

In [ ]:
def getEngine():
    global engine
    try:
        engine
    except:
        engine = getDBConnection()
    return engine

In [ ]:
driver = Chrome(webdriver_path)
driver.get("https://www.google.com/")

In [ ]:
def getDriver():
    global driver
    try:
        driver
    except:
        driver = Chrome(webdriver_path)
        driver.get("https://www.google.com/")
    return driver

In [ ]:
def fetchPage(driver, url):
	driver.get(url)
	html_page = driver.page_source
	return html_page

In [ ]:
def getArticleContent(url):
    html_content = fetchPage(driver, url)
    parser = NYTParser()
    parser.feed(html_content)
    return parser.getText()

In [ ]:
class NYTParser(HTMLParser):
    
    output = ""
    print = True
    tag = ""
    forbiddenTags = ['script', 'style', 'span', 'iframe', 'h2']
    attrs = None

    def handle_starttag(self, tag, attrs):
        #print("Encountered a start tag:", tag)
        if tag in self.forbiddenTags:
            self.print = False
        else:
            if tag == 'div':
                self.output += "\n"
            self.tag = tag
            self.attrs = attrs
        

    def handle_endtag(self, tag):
        #print("Encountered an end tag :", tag)
        if tag in self.forbiddenTags:
            self.print = True
        

    def handle_data(self, data):
        if self.print:
            if (
                len(self.attrs) == 1 and (
                    self.attrs[0][0] == 'class' or self.attrs[0][0] == 'data-rh'
                )
            ) or (
                len(self.attrs) >= 3 and self.attrs[1][0] == 'href' and self.attrs[1][1].startswith("https://")
            ):
                self.output = self.output + data + ("\n" if data.endswith("\n") else "")
                self.attrs == None
                #print(self.tag, self.attrs, data)
            else:
                #print(self.tag, data)
                pass
            
    def getText(self):
        text = self.output
        x = len(text)
        while True:
            text = text.replace("\t", "").replace("  ", " ").replace("\n\n", "")\
                .replace("\n ", "\n").replace(" \n", "\n")
            if len(text) == x:
                return text.strip()
            else:
                x = len(text)

In [ ]:
"


 

 

 
 

   "

In [ ]:


#formatted_html = format(parser.output)
#print(len(formatted_html), len(format(text)), "\n\n", formatted_html)

In [ ]:
def checkOutput(text, formatted_html):
    for sub in text.split("\n\n"):
        i = formatted_html.find(sub)
        if i < 0 or i >= len(formatted_html):
            for subsub in sub.split("\n"):
                j = formatted_html.find(subsub)
                if j < 0 or j >= len(formatted_html):
                    print(sub, "\n", "######################################")

driver.get("https://www.nytimes.com/")

Possible sources:

    "NYTimes": "https://www.nytimes.com/",
    "BBC": "https://www.bbc.co.uk/",
    "CNN": "https://edition.cnn.com/",
    "FoxNews": "https://www.foxnews.com/",
    "OAN": "https://www.oann.com/"

The Guardian, Yahoo news, Washington Post, Daily Mail, ...

https://www.4imn.com/top200/ for more

In [ ]:
def getUrlContent(source_name, params={}):
    engine = getEngine()
    sql = f"""select source_name, api_url, api_key from sources where source_name = '{source_name}'"""
    source = engine.execute(sql).fetchall()[0]
    url = source['api_url'].replace("API_KEY", source['api_key'])
    print(url)
    params = {}
    r = requests.get(url = url, params = params) 
    return r.json() 

In [ ]:
def saveData(data, name):
    with open('../raw_data/%s.txt' % name, 'w') as outfile:
        json.dump(data, outfile)

In [ ]:
def loadData(name):
    with open('../raw_data/%s.txt' % name) as json_file:
        data = json.load(json_file)
    return data

In [ ]:
def getDBConnection(schema=None):
    connectArgs = {}
    if not schema is None:
        connectArgs['options'] = '-csearch_path=' + schema
    username = "jean"
    password = ""
    host = "127.0.0.1"
    port = "5432"
    database = "media"
    postgresUrl = 'postgres://' + username + ':' + password + '@' + host + ':' + str(port) + '/' + database
    return sqlalchemy.create_engine(postgresUrl, connect_args=connectArgs)

In [ ]:
def getSourceID(name):
    engine = getEngine()
    sql = f"""select * from media.listing.sources where source_name = '{name}';"""
    sources = engine.execute(sql).fetchall()
    if len(sources) > 1:
        print("Warning: multiple sources matching name", name)
        print("Possible matches", [x['source_name'] for x in sources])
    elif len(sources) == 0:
        return None
    return str(sources[0]['source_uuid'])

In [ ]:
def insertSource(source_name, country, website_url, api_url, api_key):
    engine = getEngine()
    sql = f"""insert into media.listing.sources (source_name, country, website_url, api_url, api_key, addedat)
        values ('{source_name}', '{country}', '{website_url}', '{api_url}', '{api_key}', current_timestamp);"""
    try:
        engine.execute(sql)
    except (psycopg2.errors.UniqueViolation, sqlalchemy.exc.IntegrityError):
        print("Source already exists:", source_name)

In [ ]:
def insertArticle(
    article_url, 
    source_uuid, 
    provider_uuid, 
    title, 
    description, 
    author, 
    publishedAt, 
    updatedAt, 
    content=None):

    engine = getEngine()
    try : 
        engine.execute(f"""
            insert into articles (
                article_url, 
                source_uuid, 
                provider_uuid, 
                title, 
                description, 
                author, 
                publishedAt, 
                updatedAt)
            values (
                '{article_url}', 
                '{source_uuid}', 
                '{provider_uuid}', 
                '{title}', 
                '{description}', 
                '{author}', 
                '{publishedAt}', 
                '{updatedAt}');
                    """)
        if not description is None:
            article_uuid_sql = f"""select article_uuid from articles where article_url = '{article_url}';"""
            article_uuid = str(engine.execute(article_uuid_sql).fetchall()[0][0])
            engine.execute(f"""insert into article_contents values ('{article_uuid}', '{description}');""")
    except (psycopg2.errors.UniqueViolation, sqlalchemy.exc.IntegrityError):
        print("Article article already exists:", article_url)
        # todo manage article duplicates with difference or different sources

In [ ]:
def importNYT(data):
    source_name = "NYTimes"
    if data['status'] == 'OK':
        for i in range(data['num_results']):
            article = data['results'][i]
            source_uuid = getSourceID(source_name)
            if source_uuid == None:
                print("Source Not Found:", source_name)
            insertArticle(
                article['url'], 
                source_uuid, 
                source_uuid,
                article['title'],
                article['abstract'],
                article['byline'],
                article['published_date'],
                article['updated_date'], 
                article[])
    else:
        print("Data error:", data['status'] if 'status' in data.keys() else "JSON Error")

In [ ]:
def pipelineNYT(loadDisk=False, fetchSource=False):
    if fetchSource:
        data = getUrlContent('NYTimes')
        saveData(data, 'nyt_20200722_1000')
    if loadDisk:
        data = loadData('nyt_20200722_1000')
    importNYT(data)

In [ ]:
def getSources():
    engine = getEngine()
    page = 0
    articles = []
    while True:
        url = ('http://newsapi.org/v2/top-headlines?'
        'pageSize=100&'
        'page={page}&'
        'language=en&'
        'country=us&'
        'apiKey=e30a64cfe1734e6794bdab67106590fa')
        print(url)
        response = requests.get(url)
        json_response = response.json()
        if json_response['status'] == 'ok':
            articles.extend(json_response['articles'])
            if len(articles) == json_response['totalResults']:
                break
            elif len(articles) > json_response['totalResults']:
                print("More articles (%d) than possible to fetch (%d)" % (
                    len(articles), 
                    json_response['totalResults']))
                break
            page += 1
        else:
            print("Query status not ok")
            for k, v in json_response.items():
                print(k, v)
            break

In [ ]:
pipelineNYT(fetchSource=True)

In [ ]:
test_article_url = "https://www.nytimes.com/2020/07/21/business/economy/coronavirus-unemployment-benefits.html"
test_article_uri = "nyt://article/6fae2333-ff5a-5ac0-bcc5-34a2da53cbb3"